In [3]:
import torch
from torchvision import transforms
from torchvision.models.detection import retinanet_resnet50_fpn
from PIL import Image, ImageDraw
import os


path = "data/visdrone/VisDrone2019-DET-train/images/0000002_00005_d_0000014.jpg"
img_name = path.split('/')[-1]
classes = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',    'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',    'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',    'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',    'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',    'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',    'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',    'scissors', 'teddy bear', 'hair drier', 'toothbrush']

# Define the model
model = retinanet_resnet50_fpn(pretrained=True)

torch.save(model.state_dict(), 'pretrained_retinanet_resnet50_fpn.pth')

model.eval()

# Define the transformation

transform = transforms.Compose([transforms.ToTensor()])

# Load an image
image = Image.open(path)

# Apply the transformation
image_tensor = transform(image)

# Add a batch dimension
image_tensor = image_tensor.unsqueeze(0)

# Perform inference
with torch.no_grad():
    predictions = model(image_tensor)

# Create a drawing object
draw = ImageDraw.Draw(image)

# Threshold for filtering detections
threshold = 0.5

# Display the predictions
print(len(predictions[0]['boxes']))
for i in range(len(predictions[0]['boxes'])):
    score = predictions[0]['scores'][i]
    if score > threshold:
        box = predictions[0]['boxes'][i]
        class_id = predictions[0]['labels'][i].item()
        class_name = str(class_id)  # In RetinaNet, class labels are numeric IDs

        draw.rectangle([(box[0], box[1]), (box[2], box[3])], outline="red")
        draw.text((box[0], box[1]), f"{classes[int(class_name)-1]} - score: {score:.2f}", fill="red")

# Display the image
image.show()

# Save results
try:
    os.mkdir("results/retinaNet")
except:
    pass
image.save(f"results/retinaNet/Retina_{img_name}")

c:\Users\Nish Parikh\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Nish Parikh\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=RetinaNet_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=RetinaNet_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


242


In [4]:

import os
import random

perc = 1


# Function to calculate accuracy for an image and its annotation
def calculate_accuracy(image_path, annotation_path):
    # Load the image
    image = Image.open(image_path)

    # Apply the transformation
    image_tensor = transform(image)

    # Add a batch dimension
    image_tensor = image_tensor.unsqueeze(0)

    # Perform inference
    with torch.no_grad():
        predictions = model(image_tensor)

    # Read ground truth annotations from the text file
    with open(annotation_path, "r") as file:
        ground_truth_lines = file.readlines()

    # Parse ground truth bounding boxes
    ground_truth_boxes = []
    for line in ground_truth_lines:
        parts = line.strip().split(",")
        box = [int(parts[0]), int(parts[1]), int(parts[0]) + int(parts[2]), int(parts[1]) + int(parts[3])]
        ground_truth_boxes.append(box)

    # Initialize variables for counting true positives and total predictions
    true_positives = 0
    total_predictions = 0

    # Calculate accuracy for each predicted bounding box
    for i in range(len(predictions[0]['boxes'])):
        score = predictions[0]['scores'][i]
        if score > threshold:
            total_predictions += 1
            pred_box = predictions[0]['boxes'][i]
            pred_box = [pred_box[0].item(), pred_box[1].item(), pred_box[2].item(), pred_box[3].item()]

            # Calculate IoU with ground truth boxes
            iou_scores = []
            for gt_box in ground_truth_boxes:
                x1 = max(pred_box[0], gt_box[0])
                y1 = max(pred_box[1], gt_box[1])
                x2 = min(pred_box[2], gt_box[2])
                y2 = min(pred_box[3], gt_box[3])

                intersection = max(0, x2 - x1) * max(0, y2 - y1)
                area_pred = (pred_box[2] - pred_box[0]) * (pred_box[3] - pred_box[1])
                area_gt = (gt_box[2] - gt_box[0]) * (gt_box[3] - gt_box[1])
                union = area_pred + area_gt - intersection

                iou = intersection / union
                iou_scores.append(iou)

            # Check if IoU meets threshold for any ground truth box
            if max(iou_scores) >= perc:  # Assuming IoU threshold of 0.5 for correct detection
                true_positives += 1

    # Calculate accuracy
    accuracy = true_positives / total_predictions if total_predictions > 0 else 0
    return accuracy

# Folder paths
images_folder = "data/visdrone/VisDrone2019-DET-val/images"
annotations_folder = "data/visdrone/VisDrone2019-DET-val/annotations"



import random
from tqdm import tqdm
def calculate_mAP(accuracy_values):
    # Sort accuracy values in descending order
    sorted_accuracy = sorted(accuracy_values, reverse=True)

    # Calculate precision and recall
    precision = []
    recall = []
    true_positives = 0
    false_positives = 0
    total_positives = len(sorted_accuracy)

    for acc in sorted_accuracy:
        true_positives += acc
        false_positives += 1 - acc
        precision.append(true_positives / (true_positives + false_positives))
        recall.append(true_positives / total_positives)

    # Calculate Average Precision (AP) using precision-recall curve
    ap = 0
    for i in range(1, len(sorted_accuracy)):
        ap += (recall[i] - recall[i - 1]) * precision[i]
    print(f"||  AP%{int(perc*100)} Score = {(ap/2)*100:.2f}  ||".center(27,' '))
    print("||                       ||")

    # Calculate mean Average Precision (mAP)
    mAP = ap / len(sorted_accuracy)
    return mAP





In [5]:

accuracy_values = []
iter = 100

image_list = ['0000194_00625_d_0000122.jpg', '0000364_01373_d_0000780.jpg', '0000026_02500_d_0000029.jpg', '0000333_03137_d_0000017.jpg', '0000327_01001_d_0000716.jpg', '0000023_00000_d_0000008.jpg', '0000153_00001_d_0000001.jpg', '0000316_00001_d_0000519.jpg', '0000271_04401_d_0000394.jpg', '0000356_05097_d_0000655.jpg', '0000289_03201_d_0000827.jpg', '0000271_05001_d_0000397.jpg', '0000287_00201_d_0000760.jpg', '0000280_00001_d_0000612.jpg', '0000215_02667_d_0000262.jpg', '0000116_01059_d_0000085.jpg', '0000333_02941_d_0000016.jpg', '0000116_00819_d_0000084.jpg', '0000359_03529_d_0000712.jpg', '0000316_00401_d_0000521.jpg', '0000194_00200_d_0000120.jpg', '0000154_02001_d_0000001.jpg', '0000280_02601_d_0000625.jpg', '0000271_06601_d_0000405.jpg', '0000356_04901_d_0000654.jpg', '0000271_00401_d_0000376.jpg', '0000287_01201_d_0000765.jpg', '0000069_00001_d_0000001.jpg', '0000330_04601_d_0000823.jpg', '0000289_04601_d_0000834.jpg', '0000289_06401_d_0000843.jpg', '0000364_00589_d_0000798.jpg', '0000242_00627_d_0000003.jpg', '0000289_00401_d_0000813.jpg', '0000289_05401_d_0000838.jpg', '0000330_01401_d_0000807.jpg', '0000242_00500_d_0000002.jpg', '0000280_03001_d_0000627.jpg', '0000360_07057_d_0000749.jpg', '0000346_03921_d_0000366.jpg', '0000359_03529_d_0000712.jpg', '0000271_04801_d_0000396.jpg', '0000001_05499_d_0000010.jpg', '0000244_00001_d_0000001.jpg', '0000348_03333_d_0000423.jpg', '0000244_04400_d_0000010.jpg', '0000308_04401_d_0000327.jpg', '0000069_00001_d_0000001.jpg', '0000295_01400_d_0000028.jpg', '0000277_01401_d_0000546.jpg', '0000026_04500_d_0000033.jpg', '0000346_07057_d_0000382.jpg', '0000277_01401_d_0000546.jpg', '0000249_00001_d_0000001.jpg', '0000069_01878_d_0000005.jpg', '0000271_07001_d_0000407.jpg', '0000290_04001_d_0000867.jpg', '0000069_02480_d_0000007.jpg', '0000213_05745_d_0000247.jpg', '0000359_03333_d_0000711.jpg', '0000330_04601_d_0000823.jpg', '0000327_03801_d_0000730.jpg', '0000116_01059_d_0000085.jpg', '0000155_00401_d_0000001.jpg', '0000103_04513_d_0000034.jpg', '0000244_05900_d_0000013.jpg', '0000213_02500_d_0000240.jpg', '0000086_01954_d_0000005.jpg', '0000276_05201_d_0000533.jpg', '0000024_01543_d_0000015.jpg', '0000356_02745_d_0000643.jpg', '0000116_00351_d_0000083.jpg', '0000215_00000_d_0000256.jpg', '0000291_01401_d_0000875.jpg', '0000280_01001_d_0000617.jpg', '0000333_02941_d_0000016.jpg', '0000291_05201_d_0000893.jpg', '0000333_03529_d_0000019.jpg', '0000359_01961_d_0000707.jpg', '0000242_01139_d_0000006.jpg', '0000335_03921_d_0000063.jpg', '0000276_03801_d_0000526.jpg', '0000153_00401_d_0000001.jpg', '0000116_01059_d_0000085.jpg', '0000335_03137_d_0000059.jpg', '0000271_07001_d_0000407.jpg', '0000289_03201_d_0000827.jpg', '0000276_01401_d_0000514.jpg', '0000001_03499_d_0000006.jpg', '0000024_01000_d_0000014.jpg', '0000213_05340_d_0000246.jpg', '0000069_02163_d_0000006.jpg', '0000277_04201_d_0000559.jpg', '0000276_01401_d_0000514.jpg', '0000276_02601_d_0000520.jpg', '0000244_00500_d_0000002.jpg', '0000291_01401_d_0000875.jpg', '0000335_02745_d_0000057.jpg', '0000289_06201_d_0000842.jpg', '0000316_00201_d_0000520.jpg']


perc_vals = []
a = 0.5
for i in range(0,10):
    perc_vals.append(round(a,2))
    a+=0.05



x = len(os.listdir(images_folder))

print("RetinaNet".center(27,'='))
print("||                       ||")
for perc in perc_vals:

    for ind in tqdm(range(iter)):
        # y = random.randint(0, len(os.listdir(images_folder))-1)
        # image_name = os.listdir(images_folder)[y]
        image_name = image_list[ind]
        if image_name.endswith('.jpg'):
            image_path = os.path.join(images_folder, image_name)
            # print()
            # print(images_folder+'/'+image_name)

            annotation_name = image_name.split('.')[0] + '.txt'
            annotation_path = os.path.join(annotations_folder, annotation_name)

            accuracy = calculate_accuracy(image_path, annotation_path)
            
            accuracy_values.append(round(accuracy,4))
            

    import math
    # Calculate mAP
    # mAP = calculate_mAP(accuracy_values)
    # print(f"mAP value = : {mAP*100:.4f} ")
    # print(f"Max Accuracy  = : {max(accuracy_values)*100:.4f} ")
    # print(f"Min Accuracy  = : {min(accuracy_values)*100:.4f} ")
    print(f"Average Accuracy  = : {(sum(accuracy_values)/len(accuracy_values))*100:.4f} ")
    print()
    break
    # print(accuracy_values)


=========RetinaNet=========
||                       ||
